<a href="https://colab.research.google.com/github/ilyas-r27/Graph_Theory-Group_2/blob/main/Fleury-directed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from collections import deque
from typing import List, Tuple, Dict, Any, Optional

Node = Any
Edge = Tuple[Node, Node]
Adj  = Dict[Node, List[Node]]

In [ ]:
# Directed graph structure

def build_adj_directed(nodes: List[Node], edges: List[Edge]) -> Adj:
    adj: Adj = {v: [] for v in nodes}
    for u, v in edges:
        if u not in adj: adj[u] = []
        if v not in adj: adj[v] = []
        adj[u].append(v)
    return adj

def indegrees(adj: Adj) -> Dict[Node, int]:
    indeg = {v: 0 for v in adj}
    for u in adj:
        for v in adj[u]:
            indeg[v] += 1
    return indeg

def undirected_view(adj: Adj) -> Dict[Node, List[Node]]:
    und: Dict[Node, List[Node]] = {v: [] for v in adj}
    for u in adj:
        for v in adj[u]:
            und[u].append(v)
            und[v].append(u)
    return und

In [ ]:
# Side operations & directional reachability checks

def erase_arc(adj: Adj, u: Node, v: Node) -> None:
    adj[u].remove(v)

def add_arc(adj: Adj, u: Node, v: Node) -> None:
    adj[u].append(v)

def exists_path(u: Node, v: Node, adj: Adj) -> bool:
    if u == v:
        return True
    seen = {u}
    q = deque([u])
    while q:
        x = q.popleft()
        for y in adj[x]:
            if y == v:
                return True
            if y not in seen:
                seen.add(y)
                q.append(y)
    return False

In [ ]:
# Directed Euler rule

def has_isolated_directed(adj: Adj) -> bool:
    indeg = indegrees(adj)
    return any((len(adj[v]) + indeg[v]) == 0 for v in adj)

def connected_over_all_nodes_directed(adj: Adj) -> bool:
    m = sum(len(adj[v]) for v in adj)
    if m == 0:
        return False
    if has_isolated_directed(adj):
        return False
    und = undirected_view(adj)
    start = sorted(und.keys())[0]
    seen = {start}
    q = deque([start])
    while q:
        x = q.popleft()
        for y in und[x]:
            if y not in seen:
                seen.add(y)
                q.append(y)
    return len(seen) == len(und)

def degree_balance(adj: Adj) -> Dict[Node, int]:
    indeg = indegrees(adj)
    return {v: len(adj[v]) - indeg[v] for v in adj}

def pick_start_vertex(adj: Adj) -> Optional[Node]:
    bal = degree_balance(adj)
    starts = [v for v, b in bal.items() if b == 1]
    ends   = [v for v, b in bal.items() if b == -1]
    zeros  = [v for v, b in bal.items() if b == 0]
    if len(starts) == 1 and len(ends) == 1 and len(starts) + len(ends) + len(zeros) == len(adj):
        return starts[0]
    if len(starts) == 0 and len(ends) == 0 and len(zeros) == len(adj):
        nonempty = [v for v in adj if len(adj[v]) > 0]
        return min(nonempty) if nonempty else None
    return None

def is_cut_arc(adj: Adj, u: Node, v: Node) -> bool:
    erase_arc(adj, u, v)
    ok = exists_path(u, v, adj)
    add_arc(adj, u, v)
    return not ok

def fleury_euler_path_directed(nodes: List[Node], edges: List[Edge]) -> Optional[List[Node]]:
    adj = build_adj_directed(nodes, edges)
    if not connected_over_all_nodes_directed(adj):
        return None
    start = pick_start_vertex(adj)
    if start is None:
        return None
    cur = start
    path = [cur]
    edges_left = sum(len(adj[v]) for v in adj)
    while edges_left > 0:
        nbrs = sorted(adj[cur])
        if not nbrs:
            return None
        if len(nbrs) == 1:
            nxt = nbrs[0]
        else:
            nxt = None
            for w in nbrs:
                if not is_cut_arc(adj, cur, w):
                    nxt = w
                    break
            if nxt is None:
                nxt = nbrs[0]
        erase_arc(adj, cur, nxt)
        edges_left -= 1
        cur = nxt
        path.append(cur)
    return path

In [ ]:
# Checking result

def print_euler_result(nodes: List[Node], edges: List[Edge]) -> None:
    ans = fleury_euler_path_directed(nodes, edges)
    if ans is None:
        print("euler path not found")
    else:
        print("Eulerian path:", "-".join(map(str, ans)))

In [ ]:
print("Example A (Eulerian circuit):")
nodesA = [0, 1, 2]
edgesA = [(0,1), (1,2), (2,0)]
print_euler_result(nodesA, edgesA)

In [ ]:
print("\nExample B (Eulerian path, not circuit):")
nodesB = [0, 1, 2, 3]
edgesB = [(0,1), (1,2), (2,0), (0,3)]
print_euler_result(nodesB, edgesB)

In [ ]:
print("\nExample C (Not Eulerian: bad degree balance):")
nodesC = [0, 1, 2]
edgesC = [(0,1), (0,1), (1,2)]
print_euler_result(nodesC, edgesC)